In [1]:
# i tutaj zaczyna się przygoda!

In [2]:
## Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
import pandas as pd
import io
from scipy import stats
#import itertools
import datetime

%run funkcje.ipynb

In [3]:
# collecting data

infections = pd.read_csv('pre11_2020_infected_data.txt', sep = "\t", encoding = 'ANSI')
deaths = pd.read_csv('pre11_2020_deaths_data.txt', sep = "\t", encoding = 'ANSI')

infections2 = pd.read_csv('post11_2020_infected_data.txt', sep = "\t", encoding = 'ANSI')
deaths2 = pd.read_csv('post11_2020_deaths_data.txt', sep = "\t", encoding = 'ANSI')

population = pd.read_csv('new_ludnosc2.txt', sep = '\t', encoding = 'utf-8')

pop_area_dens = pd.read_csv('pop_area_dens.txt', sep = '\t', encoding = 'utf-8')

In [4]:
# cond = (infections['date'] == '29.05.2020') & (infections['district'].str.contains("m.", regex=False))
# print(infections.loc[cond].shape[0])
# #print_full(infections.loc[cond, 'district'])
# a = infections.loc[cond, 'district'].tolist()
# #print(a)
# #ok czyli git te co mają "m." w nazwie to miasta

In [5]:
# cond = (infections2['date'] == '27.11.2020') & (infections2['district'].str.contains("m.", regex=False))
# print(infections2.loc[cond].shape[0])
# #print_full(infections2.loc[cond, 'district'])
# b = infections2.loc[cond, 'district'].tolist()
# #print(b)
# #ok czyli git te co mają "m." w nazwie to miasta

In [6]:
# editing and sorting dataframes

infections_cities1 = infections.loc[infections['district'].str.contains("m.", regex=False)]
infections_cities2 = infections2.loc[infections2['district'].str.contains("m.", regex=False)]

deaths_cities1 = deaths.loc[deaths['district'].str.contains("m.", regex=False)]
deaths_cities2 = deaths2.loc[deaths2['district'].str.contains("m.", regex=False)]

population['new_district'] = population['district'].apply(edit_pop)
population['new_province'] = population['province'].apply(edit_pop)
population = population.drop(columns=['district'])
population = population.drop(columns=['province'])
population = population.rename(columns = {'new_district':'district'})
population = population.rename(columns = {'new_province':'province'})
population['district'] = population['district'].str.lower()
population['province'] = population['province'].str.lower()
#population = population.sort_values('district')

# tutaj edit nowych danych
pop_area_dens['new_district'] = pop_area_dens['district'].apply(edit_new)
pop_area_dens['new_province'] = pop_area_dens['province'].apply(edit_new)
pop_area_dens = pop_area_dens.drop(columns=['district'])
pop_area_dens = pop_area_dens.drop(columns=['province'])
pop_area_dens = pop_area_dens.rename(columns = {'new_district':'district'})
pop_area_dens = pop_area_dens.rename(columns = {'new_province':'province'})
pop_area_dens['district'] = pop_area_dens['district'].str.lower()
pop_area_dens['province'] = pop_area_dens['province'].str.lower()

infections['date'] = pd.to_datetime(infections['date'], format = '%d.%m.%Y') 
infections['district'] = infections['district'].apply(edit_inf)
infections['province'] = infections['province'].apply(edit_inf)
infections['district'] = infections['district'].str.lower()
infections['province'] = infections['province'].str.lower()
infections = infections[infections.district != infections.province]
infections = infections.sort_values(by=['province', 'district', 'date'])

infections_cities1['date'] = pd.to_datetime(infections_cities1['date'], format = '%d.%m.%Y') 
infections_cities1['district'] = infections_cities1['district'].apply(edit_inf)
infections_cities1['province'] = infections_cities1['province'].apply(edit_inf)
infections_cities1['district'] = infections_cities1['district'].str.lower()
infections_cities1['province'] = infections_cities1['province'].str.lower()
infections_cities1 = infections_cities1[infections_cities1.district != infections_cities1.province]
infections_cities1 = infections_cities1.sort_values(by=['province', 'district', 'date'])

infections2['date'] = pd.to_datetime(infections2['date'], format = '%d.%m.%Y') 
infections2['district'] = infections2['district'].apply(edit_inf)
infections2['province'] = infections2['province'].apply(edit_inf)
infections2['district'] = infections2['district'].str.lower()
infections2['province'] = infections2['province'].str.lower()
infections2 = infections2[infections2.district != 'sumazpowiatów']
infections2 = infections2[infections2.district != infections2.province]
infections2 = infections2.sort_values(by=['district', 'date'])

infections_cities2['date'] = pd.to_datetime(infections_cities2['date'], format = '%d.%m.%Y') 
infections_cities2['district'] = infections_cities2['district'].apply(edit_inf)
infections_cities2['province'] = infections_cities2['province'].apply(edit_inf)
infections_cities2['district'] = infections_cities2['district'].str.lower()
infections_cities2['province'] = infections_cities2['province'].str.lower()
infections_cities2 = infections_cities2[infections_cities2.district != 'sumazpowiatów']
infections_cities2 = infections_cities2[infections_cities2.district != infections_cities2.province]
infections_cities2 = infections_cities2.sort_values(by=['district', 'date'])

deaths['date'] = pd.to_datetime(deaths['date'], format = '%d.%m.%Y') 
deaths['district'] = deaths['district'].apply(edit_inf)
deaths['province'] = deaths['province'].apply(edit_inf)
deaths['district'] = deaths['district'].str.lower()
deaths['province'] = deaths['province'].str.lower()
deaths = deaths[deaths.district != deaths.province] #nie uwzględniamy województw, można zmienić, tylko, że nie mam w df danych
deaths = deaths.sort_values(by=['district', 'date']) #gęstości zaludnienia w całych województwach

deaths_cities1['date'] = pd.to_datetime(deaths_cities1['date'], format = '%d.%m.%Y') 
deaths_cities1['district'] = deaths_cities1['district'].apply(edit_inf)
deaths_cities1['province'] = deaths_cities1['province'].apply(edit_inf)
deaths_cities1['district'] = deaths_cities1['district'].str.lower()
deaths_cities1['province'] = deaths_cities1['province'].str.lower()
deaths_cities1 = deaths_cities1[deaths_cities1.district != deaths_cities1.province] #nie uwzględniamy województw, można zmienić, tylko, że nie mam w df danych
deaths_cities1 = deaths_cities1.sort_values(by=['district', 'date']) #gęstości zaludnienia w całych województwach

deaths2['date'] = pd.to_datetime(deaths2['date'], format = '%d.%m.%Y') 
deaths2['district'] = deaths2['district'].apply(edit_inf)
deaths2['province'] = deaths2['province'].apply(edit_inf)
deaths2['district'] = deaths2['district'].str.lower()
deaths2['province'] = deaths2['province'].str.lower()
deaths2 = deaths2[deaths2.district != 'sumazpowiatów']
deaths2 = deaths2[deaths2.district != deaths2.province]
deaths2 = deaths2.sort_values(by=['district', 'date'])

deaths_cities2['date'] = pd.to_datetime(deaths_cities2['date'], format = '%d.%m.%Y') 
deaths_cities2['district'] = deaths_cities2['district'].apply(edit_inf)
deaths_cities2['province'] = deaths_cities2['province'].apply(edit_inf)
deaths_cities2['district'] = deaths_cities2['district'].str.lower()
deaths_cities2['province'] = deaths_cities2['province'].str.lower()
deaths_cities2 = deaths_cities2[deaths_cities2.district != 'sumazpowiatów']
deaths_cities2 = deaths_cities2[deaths_cities2.district != deaths_cities2.province]
deaths_cities2 = deaths_cities2.sort_values(by=['district', 'date'])



In [7]:
infections['population'] = 0
infections_cities1['population'] = 0
deaths['population'] = 0
deaths_cities1['population'] = 0
infections2['population'] = 0
infections_cities2['population'] = 0
deaths2['population'] = 0
deaths_cities2['population'] = 0

infections['area_ha'] = 0
infections_cities1['area_ha'] = 0
deaths['area_ha'] = 0
deaths_cities1['area_ha'] = 0
infections2['area_ha'] = 0
infections_cities2['area_ha'] = 0
deaths2['area_ha'] = 0
deaths_cities2['area_ha'] = 0

infections['density_ppl_per_ha'] = 0
infections_cities1['density_ppl_per_ha'] = 0
deaths['density_ppl_per_ha'] = 0
deaths_cities1['density_ppl_per_ha'] = 0
infections2['density_ppl_per_ha'] = 0
infections_cities2['density_ppl_per_ha'] = 0
deaths2['density_ppl_per_ha'] = 0
deaths_cities2['density_ppl_per_ha'] = 0

In [8]:
# print_full(population)
# print_full(pop_area_dens)

In [9]:
# dodawanie populacji do df: infections i deaths + przepisywanie ostatniej valid wartości + kolumna new_infeced, new_death
last_infections = []
last_infections_cit = []
last_deaths = []
last_deaths_cit = []

# for ind in population.index:
#     inf_cond = (infections['district'] == population['district'][ind]) & (infections['province'] == population['province'][ind])
#     death_cond = (deaths['district'] == population['district'][ind]) & (deaths['province'] == population['province'][ind])
#     infections.loc[inf_cond, 'population'] = population['population'][ind]
#     deaths.loc[death_cond, 'population'] = population['population'][ind]
    
#     inf_last_valid_idx = infections.loc[inf_cond].apply(pd.Series.last_valid_index)[3]
#     death_last_valid_idx = deaths.loc[death_cond].apply(pd.Series.last_valid_index)[3]
    
#     last_infections.append(infections.loc[inf_cond, 'infected_total'].loc[inf_last_valid_idx])
#     last_deaths.append(deaths.loc[death_cond, 'dead_total'].loc[death_last_valid_idx])
    
#     infections.loc[inf_cond, 'new_infected'] = infections.loc[inf_cond, 'infected_total'].diff()
#     deaths.loc[death_cond, 'new_death'] = deaths.loc[death_cond, 'dead_total'].diff()
    
for ind in pop_area_dens.index:
    inf_cond = (infections['district'] == pop_area_dens['district'][ind]) & (infections['province'] == pop_area_dens['province'][ind])
    inf_cit_cond = (infections_cities1['district'] == pop_area_dens['district'][ind]) & (infections_cities1['province'] == pop_area_dens['province'][ind])
    death_cond = (deaths['district'] == pop_area_dens['district'][ind]) & (deaths['province'] == pop_area_dens['province'][ind])
    death_cit_cond = (deaths_cities1['district'] == pop_area_dens['district'][ind]) & (deaths_cities1['province'] == pop_area_dens['province'][ind])
    infections.loc[inf_cond, 'population'] = pop_area_dens['population'][ind]
    infections.loc[inf_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections.loc[inf_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    infections_cities1.loc[inf_cit_cond, 'population'] = pop_area_dens['population'][ind]
    infections_cities1.loc[inf_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections_cities1.loc[inf_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths.loc[death_cond, 'population'] = pop_area_dens['population'][ind]
    deaths.loc[death_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths.loc[death_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths_cities1.loc[death_cit_cond, 'population'] = pop_area_dens['population'][ind]
    deaths_cities1.loc[death_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths_cities1.loc[death_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    
    inf_last_valid_idx = infections.loc[inf_cond].apply(pd.Series.last_valid_index)[3]
    inf_cit_last_valid_idx = infections_cities1.loc[inf_cit_cond].apply(pd.Series.last_valid_index)[3]
    death_last_valid_idx = deaths.loc[death_cond].apply(pd.Series.last_valid_index)[3]
    death_cit_last_valid_idx = deaths_cities1.loc[death_cit_cond].apply(pd.Series.last_valid_index)[3]
    
    last_infections.append(infections.loc[inf_cond, 'infected_total'].loc[inf_last_valid_idx])
    if not np.isnan(inf_cit_last_valid_idx):
        last_infections_cit.append(infections_cities1.loc[inf_cit_cond, 'infected_total'].loc[inf_cit_last_valid_idx])
    else:
        last_infections_cit.append(0)
    last_deaths.append(deaths.loc[death_cond, 'dead_total'].loc[death_last_valid_idx])
    if not np.isnan(death_cit_last_valid_idx):
        last_deaths_cit.append(deaths_cities1.loc[death_cit_cond, 'dead_total'].loc[death_cit_last_valid_idx])
    else:
        last_deaths_cit.append(0)
    
    infections.loc[inf_cond, 'new_infected'] = infections.loc[inf_cond, 'infected_total'].diff()
    infections_cities1.loc[inf_cit_cond, 'new_infected'] = infections_cities1.loc[inf_cit_cond, 'infected_total'].diff()
    deaths.loc[death_cond, 'new_death'] = deaths.loc[death_cond, 'dead_total'].diff()
    deaths_cities1.loc[death_cit_cond, 'new_death'] = deaths_cities1.loc[death_cit_cond, 'dead_total'].diff()

    
    

In [10]:
# tutaj probujemy zrobic cumsum poprawnie no i dodaje populację do dataframe'u jak wyżej

# for ind in population.index:
#     inf_cond = (infections2['district'] == population['district'][ind]) & (infections2['province'] == population['province'][ind])
#     death_cond = (deaths2['district'] == population['district'][ind]) & (deaths2['province'] == population['province'][ind])
    
#     infections2.loc[inf_cond, 'population'] = population['population'][ind]
#     deaths2.loc[death_cond, 'population'] = population['population'][ind]
    
#     infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'new_infected'].cumsum()
#     deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'new_death'].cumsum()
    
#     infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'infected_total'] + last_infections[ind]
#     deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'dead_total'] + last_deaths[ind]


for ind in pop_area_dens.index:
    inf_cond = (infections2['district'] == pop_area_dens['district'][ind]) & (infections2['province'] == pop_area_dens['province'][ind])
    inf_cit_cond = (infections_cities2['district'] == pop_area_dens['district'][ind]) & (infections_cities2['province'] == pop_area_dens['province'][ind])
    death_cond = (deaths2['district'] == pop_area_dens['district'][ind]) & (deaths2['province'] == pop_area_dens['province'][ind])
    death_cit_cond = (deaths_cities2['district'] == pop_area_dens['district'][ind]) & (deaths_cities2['province'] == pop_area_dens['province'][ind])
    
    infections2.loc[inf_cond, 'population'] = pop_area_dens['population'][ind]
    infections2.loc[inf_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections2.loc[inf_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    infections_cities2.loc[inf_cit_cond, 'population'] = pop_area_dens['population'][ind]
    infections_cities2.loc[inf_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections_cities2.loc[inf_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths2.loc[death_cond, 'population'] = pop_area_dens['population'][ind]
    deaths2.loc[death_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths2.loc[death_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths_cities2.loc[death_cit_cond, 'population'] = pop_area_dens['population'][ind]
    deaths_cities2.loc[death_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths_cities2.loc[death_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    
    
    infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'new_infected'].cumsum()
    infections_cities2.loc[inf_cit_cond, 'infected_total'] = infections_cities2.loc[inf_cit_cond, 'new_infected'].cumsum()
    deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'new_death'].cumsum()
    deaths_cities2.loc[death_cit_cond, 'dead_total'] = deaths_cities2.loc[death_cit_cond, 'new_death'].cumsum()
    
    infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'infected_total'] + last_infections[ind]
    infections_cities2.loc[inf_cit_cond, 'infected_total'] = infections_cities2.loc[inf_cit_cond, 'infected_total'] + last_infections[ind]
    deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'dead_total'] + last_deaths[ind]
    deaths_cities2.loc[death_cit_cond, 'dead_total'] = deaths_cities2.loc[death_cit_cond, 'dead_total'] + last_deaths[ind]
    
    



#print(infections2.loc[infections2['date'] == "2020-11-24"])

In [11]:
#łączę dataframe'y
# infections = pd.concat([infections, infections2], verify_integrity=True)
# deaths = pd.concat([deaths, deaths2], verify_integrity=True)
# infections = pd.concat([infections, infections2], ignore_index=True)
# deaths = pd.concat([deaths, deaths2], ignore_index=True)
infections = pd.concat([infections, infections2])
infections = infections.sort_values(by=['province', 'district', 'date'])
infections = infections.reset_index(drop=True)

infections_cities = pd.concat([infections_cities1, infections_cities2])
infections_cities = infections_cities.sort_values(by=['province', 'district', 'date'])
infections_cities = infections_cities.reset_index(drop=True)

deaths = pd.concat([deaths, deaths2])
deaths = deaths.sort_values(by=['province', 'district', 'date'])
deaths = deaths.reset_index(drop=True)

deaths_cities = pd.concat([deaths_cities1, deaths_cities2])
deaths_cities = deaths_cities.sort_values(by=['province', 'district', 'date'])
deaths_cities = deaths_cities.reset_index(drop=True)



In [12]:
#   daty dla infekcji
#   04.03.2020 - 23.11.2020
#   24.11.2020 - 06.02.2022

#   daty dla smierci
#   04.03.2020 - 23.11.2020
#   24.11.2020 - 17.02.2022
# print(deaths)
#print(deaths.loc[deaths['district'] == "augustowski"])
#print(infections.loc[infections['district'] == "augustowski"])
#print(infections.loc[infections['date'] == '2022-01-06'])
#print_full(deaths.loc[deaths['district'] == "aleksandrowski"])
#print(pop_area_dens)
#print(population)
# print(infections.loc[infections['date'] == '2020-11-23'])
# print(infections.loc[infections['date'] == '2020-11-24'])
# print_full(infections.loc[infections['district'] == "leski"])

# t = infections.loc[infections['date'] == '2020-11-23', 'district'].tolist()
# s = infections.loc[infections['date'] == '2021-11-24', 'district'].tolist()
# print(compare(s, t))
#print(pop_area_dens)
# print(deaths.loc[deaths['date'] == '2020-11-23'])
# print(deaths.loc[deaths['date'] == '2021-11-24'])
# p = deaths.loc[deaths['date'] == '2020-11-23', 'district'].tolist()
# r = deaths.loc[deaths['date'] == '2021-11-24', 'district'].tolist()
# print(compare(p, r))

# zbadam zaraz dla róznych dat tak samo infekcje i śmierci

# ok, raczej reszta git
# problem opisany na discordzie #dane

In [13]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zakażeń
infections['pandemic_day'] = 0
infections['infected_mean'] = 0
window_size = 7
min_value_inf = 10  # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_inf = [] 
t = []
for ind in population.index:
    bool_infected = []
    cond = (infections['district'] == population['district'][ind]) & (infections['province'] == population['province'][ind])
    one_district = infections.loc[cond]
    if not one_district.empty:
        t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['infected_total'].rolling(window_size)
        #print(weeks)
        weeks_mean = weeks.mean()
        condition = one_district['infected_total'] > min_value_inf
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_day_inf.append(indices_list[0])
        else:
            indices_list[0] = one_district.index.to_list()[-1]
            first_day_inf.append(indices_list[0])
        
        infections.loc[cond, 'pandemic_day'] = infections.index[cond] - indices_list[0]
        infections.loc[cond, 'infected_mean'] = weeks_mean
        


In [14]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zakażeń w miastach
infections_cities['pandemic_day'] = 0
infections_cities['infected_mean'] = 0
window_size = 7
min_value_inf = 10  # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_inf = [] 
t = []
for ind in population.index:
    bool_infected = []
    cond = (infections_cities['district'] == population['district'][ind]) & (infections_cities['province'] == population['province'][ind])
    one_district = infections_cities.loc[cond]
    if not one_district.empty:
        t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['infected_total'].rolling(window_size)
        #print(weeks)
        weeks_mean = weeks.mean()
        condition = one_district['infected_total'] > min_value_inf
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_day_inf.append(indices_list[0])
        else:
            indices_list[0] = one_district.index.to_list()[-1]
            first_day_inf.append(indices_list[0])
        
        infections_cities.loc[cond, 'pandemic_day'] = infections_cities.index[cond] - indices_list[0]
        infections_cities.loc[cond, 'infected_mean'] = weeks_mean
        


In [15]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zgonów
deaths['pandemic_day'] = 0
deaths['dead_mean'] = 0
window_size = 7
min_value_deaths = 3 # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_deaths = []
t = []
for ind in population.index:
    bool_dead = []
    cond = (deaths['district'] == population['district'][ind]) & (deaths['province'] == population['province'][ind])
    one_district = deaths.loc[cond]
    if not one_district.empty:
        t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['dead_total'].rolling(window_size)
        #print(weeks)
        weeks_mean = weeks.mean()
        condition = one_district['dead_total'] > min_value_deaths
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_index = indices_list[0]
            first_day_deaths.append(first_index)
        else:
            first_index = one_district.index.to_list()[-1]
            first_day_deaths.append(first_index)

        deaths.loc[cond, 'pandemic_day'] = deaths.index[cond] - first_index
        deaths.loc[cond, 'dead_mean'] = weeks_mean


In [16]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zgonów w miastach
deaths_cities['pandemic_day'] = 0
deaths_cities['dead_mean'] = 0
window_size = 7
min_value_deaths = 3 # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_deaths = []
t = []
for ind in population.index:
    bool_dead = []
    cond = (deaths_cities['district'] == population['district'][ind]) & (deaths_cities['province'] == population['province'][ind])
    one_district = deaths_cities.loc[cond]
    if not one_district.empty:
        t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['dead_total'].rolling(window_size)
        #print(weeks)
        weeks_mean = weeks.mean()
        condition = one_district['dead_total'] > min_value_deaths
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_index = indices_list[0]
            first_day_deaths.append(first_index)
        else:
            first_index = one_district.index.to_list()[-1]
            first_day_deaths.append(first_index)

        deaths_cities.loc[cond, 'pandemic_day'] = deaths_cities.index[cond] - first_index
        deaths_cities.loc[cond, 'dead_mean'] = weeks_mean

In [17]:
#posumowac dane z powiatów do województw
# last_province_name = ""
# for ind in population.index:
#     suma = 0
#     one_province = infections.loc[(infections['province'] == population['province'][ind]) & (infections['date'] == '2020-08-25')]
#     if not one_province.empty:
#         if not (one_province['province'].iloc[0] == last_province_name):
#             last_province_name = one_province['province'].iloc[0]
#             #print(one_province)
#             suma += one_province['population'].sum()
#             whole_province = one_province.loc[one_province['province'] == one_province['district']]
#             suma /= 2
#             print(str(one_province['province'].iloc[0]))
#             print("Z danych: "+ str(whole_province['population'].iloc[0]))
#             print("Posumowane: " + str(suma))
#             print()
        

In [18]:
# print(infections_cities.loc[infections_cities['district'] == 'jeleniagóra'])

In [19]:
#print_full(infections.loc[infections['district'] == 'warszawa', ('date', 'infected_mean', 'infected_total')])

In [20]:
# Okej, obczaiłem. Jest rozbieżność pomiędzy danymi dot. zakażeń w tym pliku excelowskim, a tymi w tekstowym. 
# W tekstowym, skąd brałem dane nie ma wszystkich powiatów, np. w województwie kujawsko-pomorskim jest tylko powiat aleksanrowsk
# 